In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import datetime

In [5]:
def get_all_data():
    # Parse data.json file
    data = requests.get('https://api.covid19india.org/data.json').json()

    # Create data frame for testing data
    df_tested = pd.DataFrame.from_dict(data['tested'])
    df_tested = df_tested[np.logical_not(df_tested['source'] == '')]
    df_tested.reset_index(inplace=True, drop=True)

    # Create data frame for statewise data
    for i, x in enumerate(data['statewise']):
        delta_dict = dict([('delta_'+k, v) for k, v in x['delta'].items()])
        data['statewise'][i].update(delta_dict)
        del data['statewise'][i]['delta']
    df_statewise = pd.DataFrame.from_dict(data['statewise'])

    # Create data frame for time series data
    df_india_time_series = pd.DataFrame.from_dict(data['cases_time_series'])
    df_india_time_series['date'] = pd.to_datetime([datetime.datetime.strptime(x[:6]+ ' 2020', '%d %b %Y') for x in df_india_time_series['date']])
    
    # Parse state_district_wise.json file
    data = requests.get('https://api.covid19india.org/state_district_wise.json').json()
    states = data.keys()
    for state in states:
        for district, district_dict in data[state]['districtData'].items():
            delta_dict = dict([('delta_'+k, v) for k, v in district_dict['delta'].items()])
            data[state]['districtData'][district].update(delta_dict)
            del data[state]['districtData'][district]['delta']

    columns = ['state', 'district', 'confirmed', 'delta_confirmed', 'lastupdatedtime']
    df_districtwise = pd.DataFrame(columns=columns)
    for state in states:
        df = pd.DataFrame.from_dict(data[state]['districtData']).T.reset_index()
        df.columns = ['district', 'confirmed', 'delta_confirmed', 'lastupdatedtime']
        df['state'] = state
        df = df[columns]
        df_districtwise = pd.concat([df_districtwise, df], ignore_index=True)
        
    # Parse raw_data.json file
    # Create dataframe for raw history
    data = requests.get('https://api.covid19india.org/raw_data.json').json()
    df_raw_data = pd.DataFrame.from_dict(data['raw_data'])
    df_raw_data = df_raw_data[np.logical_not(df_raw_data['source1'] == '')]

    # Parse travel_history.json file
    # Create dataframe for travel history
    data = requests.get('https://api.covid19india.org/travel_history.json').json()
    df_travel_history = pd.DataFrame.from_dict(data['travel_history'])
    
    return df_tested, df_statewise, df_india_time_series, df_districtwise, df_raw_data, df_travel_history

In [6]:
df_tested, df_statewise, df_india_time_series, df_districtwise, df_raw_data, df_travel_history = get_all_data()

In [7]:
data = requests.get('https://api.rootnet.in/covid19-in/stats/history').json()

In [30]:
columns = ['confirmedCasesForeign', 'confirmedCasesIndian', 'deaths', 'discharged', 'loc', 'date']
df = pd.DataFrame(columns=columns)
for i, data_json in enumerate(data['data']):
    df_temp = pd.DataFrame.from_dict(data['data'][i]['regional'])
    df_temp['date'] = data['data'][i]['day']
    df = pd.concat([df, df_temp])
    
df['confirmedCases'] = df['confirmedCasesForeign'] + df['confirmedCasesIndian']
df

,confirmedCasesForeign,confirmedCasesIndian,deaths,discharged,loc,date,confirmedCases
0,0,4,0,0,Delhi,2020-03-10,4
1,14,0,0,0,Haryana,2020-03-10,14
2,0,1,0,0,Karnataka,2020-03-10,1
3,0,9,0,0,Kerala,2020-03-10,9
4,0,2,0,0,Maharashtra,2020-03-10,2
5,0,1,0,0,Punjab,2020-03-10,1
6,2,0,0,0,Rajasthan,2020-03-10,2
7,0,1,0,0,Tamil Nadu,2020-03-10,1
8,0,1,0,0,Telengana,2020-03-10,1
9,0,1,0,0,Jammu and Kashmir,2020-03-10,1


In [31]:
df[df['loc'] == 'Maharashtra']

,confirmedCasesForeign,confirmedCasesIndian,deaths,discharged,loc,date,confirmedCases
4,0,2,0,0,Maharashtra,2020-03-10,2
4,0,2,0,0,Maharashtra,2020-03-11,2
4,0,11,0,0,Maharashtra,2020-03-12,11
5,0,14,0,0,Maharashtra,2020-03-13,14
5,0,14,0,0,Maharashtra,2020-03-14,14
5,0,32,0,0,Maharashtra,2020-03-15,32
5,0,32,0,0,Maharashtra,2020-03-16,32
5,3,36,1,0,Maharashtra,2020-03-17,39
5,3,39,1,0,Maharashtra,2020-03-18,42
6,3,44,1,0,Maharashtra,2020-03-19,47
